# Bia-bob
This notebook is used to generate a python-file that serves as interoperability connector to [bia-bob](https://github.com/haesleinhuepf/bia-bob). You need an OpenAI account to run this notebook. Note: It uses chatGPT in a loop and executing this notebook costs some cents.

In [1]:
import pyclesperanto as cle
import openai
cle.__version__, openai.__version__

('0.13.1', '1.43.0')

In [2]:
def prompt(message:str, model="gpt-4o-2024-08-06"):
    """A prompt helper function that sends a message to openAI
    and returns only the text response.
    """
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": message}]
    )
    return response.choices[0].message.content

In [3]:
all_suggested_functions = list(cle.operations(['bia-bob-suggestion']).keys())
len(all_suggested_functions)

87

In [4]:
all_suggested_functions

['absolute',
 'absolute_difference',
 'binary_and',
 'binary_edge_detection',
 'binary_not',
 'binary_or',
 'binary_subtract',
 'binary_xor',
 'bottom_hat',
 'bottom_hat_sphere',
 'centroids_of_labels',
 'closing',
 'closing_labels',
 'closing_sphere',
 'combine_labels',
 'concatenate_along_x',
 'concatenate_along_y',
 'concatenate_along_z',
 'connected_component_labeling',
 'connected_components_labeling',
 'count_touching_neighbors',
 'detect_label_edges',
 'difference_of_gaussian',
 'dilate',
 'dilate_labels',
 'dilate_sphere',
 'divide_by_gaussian_background',
 'erode',
 'erode_sphere',
 'eroded_otsu_labeling',
 'exclude_labels_on_edges',
 'exclude_large_labels',
 'exclude_small_labels',
 'extend_labeling_via_voronoi',
 'gauss_otsu_labeling',
 'gaussian_blur',
 'generate_distance_matrix',
 'generate_touch_matrix',
 'label',
 'label_spots',
 'labelled_spots_to_pointlist',
 'laplace',
 'laplace_box',
 'laplace_diamond',
 'mask',
 'mask_label',
 'masked_voronoi_labeling',
 'maximum_im

In [5]:
selected_suggested_functions = {}
for f in all_suggested_functions:
    func = getattr(cle, f)
    if func.__name__ != f:
        print(f"Ignoring alias '{f}'")
    else:
        selected_suggested_functions[f] = getattr(cle, f)    

Ignoring alias 'label'


In [6]:
def get_function_signature(func):
    import inspect
    
    function_object = func
    signature_object = inspect.signature(function_object)
    signature_string = "cle." + func.__name__ + str(signature_object)
    signature_string = signature_string.replace("Union[numpy.ndarray, pyclesperanto._pyclesperanto._Array], output_image: Union[numpy.ndarray, pyclesperanto._pyclesperanto._Array, NoneType]", "ndarray")    
    signature_string = signature_string.replace(", device: Optional[pyclesperanto._pyclesperanto._Device] = None", "")
    signature_string = signature_string.replace("Union[numpy.ndarray, pyclesperanto._pyclesperanto._Array]", "ndarray")
    signature_string = signature_string.replace(" = None", "")
    return signature_string

all_descriptions = []
for name, func in selected_suggested_functions.items():
    
    description = prompt(f"""
    # Writing a quick-guide for a python function
    You will turn a function signature and a docstring into a single-liner to guide users how to use that function.
    
    Shorten the following docstring into a single bullet point:

    ```
    {func.__doc__}
    ```
    
    Shorten it to a single bullet point explaining what the function does without mentioning the function name and without phrases such as `use this function`.

    ## Examples

    * compute the absolute value of every individual pixel in an image

    * determine the absolute difference pixel by pixel between two images
    
    * add a scalar value to all pixels of an image

    ## Your task
    
    What is a good short bullet point for the documentation given above?
    """).replace("If you want to ", "* ")
    description = description + "\n" + get_function_signature(func) + "\n\n"
    print(description)
    
    all_descriptions.append(description)

#print(all_descriptions[:1000])

- Compute the absolute value of each pixel in an input image, optionally specifying an output image and device for processing.
cle.absolute(input_image: ndarray) -> ndarray


- Calculate the pixel-by-pixel absolute difference between two images, optionally storing the result in an output image and selecting a device for operation.
cle.absolute_difference(input_image0: ndarray, input_image1: ndarray) -> ndarray


* Generate a binary image by applying the binary AND operator to corresponding pixels of two input images, considering all non-zero pixels as 1.
cle.binary_and(input_image0: ndarray, input_image1: ndarray) -> ndarray


* Identify and highlight edge pixels of binary objects, setting them to 1 in the output image on a specified device.
cle.binary_edge_detection(input_image: ndarray) -> ndarray


* Convert an image to a binary image by negating all non-zero pixels using the binary NOT operator.
cle.binary_not(input_image: ndarray) -> ndarray


* Combine two images pixel-wise using

In [7]:
descriptions = []
for bullet_point in all_descriptions:
    if bullet_point[0:2] in ['- ', '* ']:
        bullet_point = bullet_point[2:]

    descriptions.append("* " + bullet_point)


In [8]:
text = "".join(descriptions).replace('"', "'")
text = "    " + text.replace("\n", "\n    ")
text = f'''
def list_bia_bob_plugins():
    """List of function hints for bia_bob"""
    return """
    ## pyclesperanto
    pyclesperanto is a Python library for GPU-accelerated image processing and analysis. 
    To use it, you need to import it:
    ```
    import pyclesperanto as cle
    ```
    
{text}"""
'''

with open("../../pyclesperanto/_bia_bob_plugins.py", 'w') as file:
    file.write(text)